In [1]:
from data_loading.data_loader import load_data

In [2]:
data = load_data()

/home/indika/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [3]:
import torch
import numpy as np

In [4]:
np.random.shuffle(data)

In [5]:
X = []
y = []
for d in data:
    X.append(d[0])
    y.append(d[1])
VAL_SPLIT = 0.25
VAL_SPLIT = len(X) * VAL_SPLIT
VAL_SPLIT = int(VAL_SPLIT)
X_train = X[:-VAL_SPLIT]
y_train = y[:-VAL_SPLIT]
X_test = X[-VAL_SPLIT:]
y_test = y[-VAL_SPLIT:]
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))
X_train = torch.from_numpy(np.array(X_train))
y_train = torch.from_numpy(np.array(y_train))
X_test = torch.from_numpy(np.array(X_test))
y_test = torch.from_numpy(np.array(y_test))
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

6276
6276
2091
2091
6276
6276
2091
2091


## Modelling

In [6]:
from torchvision import models

In [7]:
device = torch.device('cuda')

In [8]:
import torch.nn as nn

In [9]:
# model = models.resnet18(pretrained=True).to(device)
# inf = model.fc.in_features
# model.fc = nn.Linear(inf,2)
from models.baseline_model import BaseLine_Model
model = BaseLine_Model().to(device)

In [10]:
model = model.to(device)

In [11]:
criterion = nn.CrossEntropyLoss()

In [12]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)

In [13]:
BATCH_SIZE = 32

In [14]:
EPOCHS = 100

In [15]:
import wandb

In [16]:
PROJECT_NAME = 'Face-Mask-Detection'

In [17]:
from tqdm import tqdm

In [20]:
wandb.init(project=PROJECT_NAME,name='test')
for _ in tqdm(range(EPOCHS),leave=False):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        preds = model(X_batch)
        preds.to(device)
        print(preds.shape)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'loss':loss.item()})
wandb.finish()

torch.Size([3200, 2])


ValueError: Expected input batch_size (3200) to match target batch_size (32).

In [19]:
for index in range(len(preds)):
    print(torch.argmax(torch.round(preds)[index]))
    print(y_batch[index])
    print('\n')

tensor(1, device='cuda:0')
tensor(1, device='cuda:0')


tensor(1, device='cuda:0')
tensor(1, device='cuda:0')


tensor(1, device='cuda:0')
tensor(1, device='cuda:0')


tensor(0, device='cuda:0')
tensor(0, device='cuda:0')


